# GPT 5 Testing
## Basic Image Preprocessing using GPT5

## Plan: Preprocess SIPaKMeD cropped images
We will:
- Use the cropped class folders in `Dataset/Raw Dataset/SipakMed Dataset/*`.
- Standardize each image to a square canvas and resize to 256x256.
- Apply gentle denoising (median), gray-world white balance, and CLAHE on luminance for contrast.
- Optionally apply light unsharp masking.
- Preserve class subfolders and save as PNG.

Output will be written to: `Dataset/Preprocessed_SipakMed_256_GW_CLAHE_USM/`

In [4]:
# Imports and configuration
from pathlib import Path
import os
import sys
import math
import gc
from concurrent.futures import ThreadPoolExecutor, as_completed

import numpy as np

try:
    import cv2  # Preferred for image ops
except Exception:
    cv2 = None

try:
    from PIL import Image, ImageFilter, ImageOps
except Exception as e:
    raise RuntimeError("Pillow is required. Please install with `pip install pillow`.\n" + str(e))

try:
    from tqdm import tqdm
except Exception:
    def tqdm(x, **kwargs):
        return x

# Absolute paths (edit if needed)
DATASET_BASE = Path(r"c:/Meet/Projects/Project_8_Phoenix_Cervical Cancer Image Classification/Project-Phoenix/Dataset")
INPUT_DIR = DATASET_BASE / "Raw Dataset" / "SipakMed Dataset"
# New output folder name
OUTPUT_DIR = DATASET_BASE / "Preprocessed_SipakMed_256_GW_CLAHE_USM"

# Preprocessing parameters
TARGET_SIZE = 256  # final square size
MEDIAN_KSIZE = 3   # gentle denoising
CLAHE_CLIP_LIMIT = 1.3
CLAHE_TILE_GRID = (8, 8)
APPLY_UNSHARP = True
UNSHARP_AMOUNT = 0.35  # 0..1 typical
UNSHARP_RADIUS = 0.9
# Blend factor for white balance (0=no WB, 1=full WB)
WB_BLEND = 0.7

# Threading
MAX_WORKERS = max(4, os.cpu_count() or 4)

# Valid image extensions
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"}

print(f"Input:  {INPUT_DIR}")
print(f"Output: {OUTPUT_DIR}")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

Input:  c:\Meet\Projects\Project_8_Phoenix_Cervical Cancer Image Classification\Project-Phoenix\Dataset\Raw Dataset\SipakMed Dataset
Output: c:\Meet\Projects\Project_8_Phoenix_Cervical Cancer Image Classification\Project-Phoenix\Dataset\Preprocessed_SipakMed_256_GW_CLAHE_USM


In [5]:
# Helpers: IO and preprocessing primitives

def is_image_file(p: Path) -> bool:
    return p.suffix.lower() in IMG_EXTS


def read_image_rgb(path: Path) -> np.ndarray:
    """Read image as RGB uint8 array. Handles alpha by compositing on white."""
    with Image.open(path) as im:
        if im.mode in ("RGBA", "LA"):
            bg = Image.new("RGBA", im.size, (255, 255, 255, 255))
            im = Image.alpha_composite(bg, im.convert("RGBA")).convert("RGB")
        elif im.mode != "RGB":
            im = im.convert("RGB")
        arr = np.array(im, dtype=np.uint8)
    return arr


def save_png_rgb(path: Path, arr: np.ndarray):
    path.parent.mkdir(parents=True, exist_ok=True)
    img = Image.fromarray(arr.astype(np.uint8), mode="RGB")
    img.save(path, format="PNG", optimize=True)


def compute_border_fill_color(img_rgb: np.ndarray) -> tuple:
    """Estimate a pleasant pad color using border median."""
    h, w, _ = img_rgb.shape
    border = np.concatenate([
        img_rgb[0, :, :], img_rgb[-1, :, :],
        img_rgb[:, 0, :], img_rgb[:, -1, :]
    ], axis=0)
    med = np.median(border, axis=0)
    return tuple(int(x) for x in med)


def resize_with_padding(img_rgb: np.ndarray, target: int) -> np.ndarray:
    h, w, _ = img_rgb.shape
    scale = min(target / h, target / w)
    nh, nw = max(1, int(round(h * scale))), max(1, int(round(w * scale)))

    if cv2 is not None:
        resized = cv2.resize(img_rgb, (nw, nh), interpolation=cv2.INTER_AREA)
    else:
        resized = np.array(Image.fromarray(img_rgb).resize((nw, nh), Image.Resampling.LANCZOS))

    pad_color = compute_border_fill_color(img_rgb)
    canvas = np.full((target, target, 3), pad_color, dtype=np.uint8)
    y0 = (target - nh) // 2
    x0 = (target - nw) // 2
    canvas[y0:y0+nh, x0:x0+nw] = resized
    return canvas


def median_denoise(img_rgb: np.ndarray, ksize: int = 3) -> np.ndarray:
    if ksize and ksize >= 3:
        if cv2 is not None:
            return cv2.medianBlur(img_rgb, ksize)
        else:
            return np.array(Image.fromarray(img_rgb).filter(ImageFilter.MedianFilter(size=ksize)))
    return img_rgb


def gray_world_white_balance(img_rgb: np.ndarray) -> np.ndarray:
    # Gray-world assumption with blending
    img = img_rgb.astype(np.float32)
    means = img.reshape(-1, 3).mean(axis=0)
    gray = float(means.mean()) + 1e-6
    gains = gray / (means + 1e-6)
    corrected = np.clip(img * gains, 0, 255)
    blended = (1.0 - WB_BLEND) * img + WB_BLEND * corrected
    return blended.astype(np.uint8)


def clahe_on_luminance(img_rgb: np.ndarray, clip: float = 2.0, tile=(8, 8)) -> np.ndarray:
    if cv2 is None:
        # Fallback: mild autocontrast on Y channel approximation
        im = Image.fromarray(img_rgb, 'RGB').convert('YCbCr')
        y, cb, cr = im.split()
        y = ImageOps.autocontrast(y, cutoff=1)
        im = Image.merge('YCbCr', (y, cb, cr)).convert('RGB')
        return np.array(im)
    lab = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=CLAHE_CLIP_LIMIT, tileGridSize=CLAHE_TILE_GRID)
    l2 = clahe.apply(l)
    lab2 = cv2.merge((l2, a, b))
    rgb = cv2.cvtColor(lab2, cv2.COLOR_LAB2RGB)
    return rgb


def unsharp_mask(img_rgb: np.ndarray, radius: float = 1.0, amount: float = 0.6) -> np.ndarray:
    if amount <= 0:
        return img_rgb
    if cv2 is not None:
        # Convert radius (sigma) to kernel size; ensure odd
        sigma = max(0.1, float(radius))
        k = int(round(sigma * 6)) | 1
        blur = cv2.GaussianBlur(img_rgb, (k, k), sigmaX=sigma)
        # img * (1+amount) - blur * amount
        sharp = cv2.addWeighted(img_rgb, 1.0 + amount, blur, -amount, 0)
        return np.clip(sharp, 0, 255).astype(np.uint8)
    else:
        pil = Image.fromarray(img_rgb)
        pil = pil.filter(ImageFilter.UnsharpMask(radius=radius, percent=int(amount*200), threshold=2))
        return np.array(pil)


def pipeline(img_rgb: np.ndarray) -> np.ndarray:
    # 1) Gentle denoise
    x = median_denoise(img_rgb, MEDIAN_KSIZE)
    # 2) White balance (gray-world)
    x = gray_world_white_balance(x)
    # 3) Local contrast (CLAHE on luminance)
    x = clahe_on_luminance(x)
    # 4) Resize with padding to square
    x = resize_with_padding(x, TARGET_SIZE)
    # 5) Optional light sharpening
    if APPLY_UNSHARP:
        x = unsharp_mask(x, UNSHARP_RADIUS, UNSHARP_AMOUNT)
    return x

In [6]:
# Build file list and run preprocessing

def find_images(base: Path):
    files = []
    for p in base.rglob('*'):
        if p.is_file() and is_image_file(p):
            files.append(p)
    return files


def dest_from_src(src: Path) -> Path:
    rel = src.relative_to(INPUT_DIR)
    # Keep class folders, force .png extension
    return OUTPUT_DIR / rel.with_suffix('.png')


def process_one(src: Path) -> tuple:
    dst = dest_from_src(src)
    try:
        arr = read_image_rgb(src)
        out = pipeline(arr)
        save_png_rgb(dst, out)
        return (True, src, None)
    except Exception as e:
        return (False, src, str(e))


all_imgs = find_images(INPUT_DIR)
print(f"Found {len(all_imgs)} images under {INPUT_DIR}")

ok = 0
fail = 0
errors = []

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
    futures = [ex.submit(process_one, p) for p in all_imgs]
    for fut in tqdm(as_completed(futures), total=len(futures), desc='Preprocessing'):
        success, src, err = fut.result()
        if success:
            ok += 1
        else:
            fail += 1
            errors.append((src, err))

print(f"Done. Saved: {ok}, Failed: {fail}")
if fail:
    print("Sample errors:")
    for s, e in errors[:5]:
        print("-", s, "->", e)

# Free memory
gc.collect()

Found 5015 images under c:\Meet\Projects\Project_8_Phoenix_Cervical Cancer Image Classification\Project-Phoenix\Dataset\Raw Dataset\SipakMed Dataset


Preprocessing: 100%|██████████| 5015/5015 [01:44<00:00, 48.06it/s] 

Done. Saved: 5015, Failed: 0


9